<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
# TODO - Words, words, mere words, no matter from the heart.

In [2]:
!wget https://www.gutenberg.org/files/100/100-0.txt

--2019-12-16 19:40:07--  https://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5777367 (5.5M) [text/plain]
Saving to: ‘100-0.txt.10’

100-0.txt.10        100%[===================>]   5.51M  1.66MB/s    in 3.3s    

2019-12-16 19:40:10 (1.66 MB/s) - ‘100-0.txt.10’ saved [5777367/5777367]



In [3]:
#data = list()

#with open ('100-0.txt', 'r') as f:
#    data.append(f.read())

In [4]:
#len(data)

In [5]:
#import re
#import string

#re.sub(r'[^a-zA-Z ^0-9]', '', full_text)

In [6]:
#data1 = data[0].replace('\n', '')

In [7]:
#data1 = re.sub(r'[^a-zA-Z ^0-9]', '', data1)

In [8]:
import re
import string
import numpy as np
import random
import sys

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

#re.sub(r'[^a-zA-Z ^0-9]', '', full_text)

In [9]:
data = list()

with open ('100-0.txt', 'r') as f:
    data.append(f.read())

len(data)



import re
import string

#re.sub(r'[^a-zA-Z ^0-9]', '', full_text)
# debate over taking out spacing

data1 = data[0].replace('\n', '')
data1 = data[0].replace('\t', '')

# character filter
#data1 = re.sub(r'[^a-zA-Z^0-9]', '', data1)

big_string = " ".join(data1)

big_string = data1
character = list(set(big_string))

len(character)

106

In [10]:
# so far, all spaces and irregular characters have been removed

In [11]:
char_int = {character:integer for
           integer, character in enumerate(character)}

int_char = {integer:character for
           integer, character in enumerate(character)}

In [12]:
# create the sequence of data... 

maxlen = 32
step = 5
encoded = [char_int[c] for c in big_string]
sequences = [] # or list()
next_character = []


In [13]:
for i in range (0, len(encoded) - maxlen, step):
    # the 127 characters
    sequences.append(encoded[i: i + maxlen])
    # the 128th character
    next_character.append(encoded[i + maxlen])

In [14]:
X = np.zeros((len(sequences), maxlen, len(character)), dtype=np.bool)
y = np.zeros((len(sequences), len(character)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, characters in enumerate(sequence):
        X[i,t,characters] = 1
    
    y[i, next_character[i]] = 1

In [15]:
X.shape

(1114624, 32, 106)

In [16]:
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen, len(character))))
model.add(Dense(len(character), activation='softmax'))

          
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [17]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [18]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    #start_index = np.random.randint(0, len(sequence) - maxlen - 1)
    start_index = random.randint(0, len(sequence) - maxlen)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)
        generated = ''
        sentence = big_string[start_index: start_index + maxlen]
        generated = generated + sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(character)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_characters = int_char[next_index]
            sentence = sentence[1:] + next_characters
            sys.stdout.write(next_characters)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [19]:
'''
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(sequence) - maxlen)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = big_string[start_index: start_index + maxlen]
        generated = generated + sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(characters)))
            for t, character in enumerate(sentence):
                x_pred[0, t, char_int[character]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_characters = int_char[next_index]

            sentence = sentence[1:] + next_characters

            sys.stdout.write(next_characters)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
'''

'\ndef on_epoch_end(epoch, _):\n    # Function invoked at end of each epoch. Prints generated text.\n    print()\n    print(\'----- Generating text after Epoch: %d\' % epoch)\n\n    start_index = random.randint(0, len(sequence) - maxlen)\n    for diversity in [0.2, 0.5, 1.0, 1.2]:\n        print(\'----- diversity:\', diversity)\n\n        generated = \'\'\n        sentence = big_string[start_index: start_index + maxlen]\n        generated = generated + sentence\n        print(\'----- Generating with seed: "\' + sentence + \'"\')\n        sys.stdout.write(generated)\n\n        for i in range(400):\n            x_pred = np.zeros((1, maxlen, len(characters)))\n            for t, character in enumerate(sentence):\n                x_pred[0, t, char_int[character]] = 1.\n\n            preds = model.predict(x_pred, verbose=0)[0]\n            next_index = sample(preds, diversity)\n            next_characters = int_char[next_index]\n\n            sentence = sentence[1:] + next_characters\n\n   

In [20]:

'''
#class
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(big_string) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = big_string[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
'''

'\n#class\ndef on_epoch_end(epoch, _):\n    # Function invoked at end of each epoch. Prints generated text.\n    print()\n    print(\'----- Generating text after Epoch: %d\' % epoch)\n\n    start_index = np.random.randint(0, len(big_string) - maxlen - 1)\n    for diversity in [0.2, 0.5, 1.0, 1.2]:\n        print(\'----- diversity:\', diversity)\n\n        generated = \'\'\n        sentence = big_string[start_index: start_index + maxlen]\n        generated += sentence\n        print(\'----- Generating with seed: "\' + sentence + \'"\')\n        sys.stdout.write(generated)\n\n        for i in range(400):\n            x_pred = np.zeros((1, maxlen, len(chars)))\n            for t, char in enumerate(sentence):\n                x_pred[0, t, char_indices[char]] = 1.\n\n            preds = model.predict(x_pred, verbose=0)[0]\n            next_index = sample(preds, diversity)\n            next_char = indices_char[next_index]\n\n            sentence = sentence[1:] + next_char\n\n            sys.

In [21]:
model.fit(X, y,
          batch_size=512,
          epochs=1,
          callbacks=[print_callback])

Train on 1114624 samples
1114112/1114624 [============================>.] - ETA: 0s - loss: 1.8276
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "﻿
Project Gutenberg’s The Comple"
﻿
Project Gutenberg’s The Compleanio.

CROMIO.
The will see the strange of the stands.
    The will be the send the sons the strength the strength,
    The sent the senst the sentle shall the shall be see the wifter are the wiftle and the straight.
    The world to the wifter the sentle of the world to the world the to the hands and the son the send the send the sentlemen and the sentle same the grave the sons all the wifted so 
----- diversity: 0.5
----- Generating with seed: "﻿
Project Gutenberg’s The Comple"
﻿
Project Gutenberg’s The Compleuaral she are not the grace in thee,
The sander of the hearth and the can may have a with heart,
    With me to be the most now a show a timest, sir,
    Thear mean to him, and but may heaven be deeding furth.
    When the dreather

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN